## 3. Let's have a look at the NeXtSIM-DG outputs: 

In this notebook you will: 
* 3.1 Load python libraries,
* 3.2 Load the land/sea mask and initial conditions,
* 3.3 Plot land/sea mask and SST initial conditions,
* 3.4 Load and plot the  model outputs files.

---
## 3.1 Load python libraries

In [ ]:
## standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# plot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from matplotlib import animation
from IPython.display import HTML

import datetime, time

#JUPYTER notebook magics
%matplotlib inline 

# ---------------------
# Some functions
def printdate(unixt=0):
    """ Print string date from unix time """
    #unixt = float(outputs.time[3].values)
    dt = datetime.datetime.fromtimestamp(unixt)
    printdate = str(dt.year)+"-"+str(dt.month).zfill(2)+"-"+str(dt.day).zfill(2)+":"+str(dt.hour).zfill(2)+":00"
    return printdate

---
## 3.2. Load the land/sea mask and initial conditions
from file : `init_25km_NH.nc`.

* Path where the model  files are:

In [ ]:
dirwo = "/home/jovyan/work/"

* read the initial data:

In [ ]:
datinit= xr.open_dataset(dirwo+'/init_25km_NH.nc',group='data')
datinit

_Note:_ that the "group" option is necessary to read the files (netcdf4 format).

---
## 3.3 Plot land/sea mask

* Land sea mask:

In [ ]:
fig,(ax) = plt.subplots(1, 1, figsize=[9, 6],facecolor='w')
p = ax.pcolormesh(datinit.mask,cmap='seismic_r')
plt.colorbar(p,label='LAND/SEA MASK')
plt.show
plt.close

---
## 3.4 Load and plot the  model outputs files 


In [ ]:
outputs= xr.open_dataset(dirwo+'/output_simudyn_5d.nc',group='data')
outputs

### 3.4.aAnimate the evolution of the simulated sea ice thickness 

In [ ]:
# Number of timesteps in the output file
NTIME = outputs.time.size

# select colormap and set gray color for missing values (i.e. for land)
cmap = cm.inferno
cmap.set_bad('gray',1.)

# select sea ice height
data2plt = outputs.hice
# mask  land so that it appears gray
data2plt_masked = data2plt.where(datinit.mask!=0)


fig,(ax) = plt.subplots(1, 1, figsize=[9, 6],facecolor='w')

#plot
it=0
p = ax.pcolormesh(data2plt_masked.isel(time=it),vmin=0,vmax=3,cmap=cmap)

# add date
da = printdate(unixt=float(data2plt_masked.time[it].values))
d = ax.annotate(da, xy=(4, 113),  fontsize=12, color="w")

# add colorbar
plt.colorbar(p,label='sea ice height (m)',extend='both')


def animate(it):
    p.set_array(data2plt_masked.isel(time=it))
    da = printdate(unixt=float(data2plt_masked.time[it].values))
    d = ax.annotate(da, xy=(4, 113),  fontsize=12, color="w",backgroundcolor='gray')
    return p,

def init():
    p.set_array(data2plt_masked.isel(time=0))
    return p,

ani = animation.FuncAnimation(fig=fig,
                              func=animate,
                              init_func=init,
                              frames=NTIME,
                              interval=100,
                              blit=True)

HTML(ani.to_jshtml())

### 3.4.b Animate the evolution of the simulated sea ice concentration 

In [ ]:
# Number of timesteps in the output file
NTIME = outputs.time.size


# select colormap and set gray color for missing values (i.e. for land)
cmap = cm.bone
cmap.set_bad('gray',1.)
norm = mcolors.PowerNorm(gamma=4.5,vmin=0, vmax=1)


# select sea ice concentration
data2plt = outputs.cice

# mask  land so that it appears gray
data2plt_masked = data2plt.where(datinit.mask!=0)

# create first figure
fig,(ax) = plt.subplots(1, 1, figsize=[7.5, 6],facecolor='w')

it=0

# plot sea ice field
p = ax.pcolormesh(data2plt_masked.isel(time=it),cmap=cmap,norm=norm)

# date label on top of plot
da = printdate(unixt=float(data2plt_masked.time[it].values))
d = ax.annotate(da, xy=(4, 113),  fontsize=12, color="w")

# colorbar
cblev=[0,0.7,0.8,0.85,0.9,0.95,0.99]
axins1 = inset_axes(ax,
                        height="10%",  # height : 5%
                        width="50%",
                        bbox_to_anchor=(0.08, 0.92,0.9,0.2),
                        bbox_transform=ax.transAxes,
                        borderpad=0)

cb = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=axins1, orientation='horizontal',ticks=cblev,label='sea ice concentration')




def animate(it):
    p.set_array(data2plt_masked.isel(time=it))
    da = printdate(unixt=float(data2plt_masked.time[it].values))
    d = ax.annotate(da, xy=(4, 113),  fontsize=12, color="w",backgroundcolor='gray')
    return p,

def init():
    p.set_array(data2plt_masked.isel(time=0))
    return p,

ani = animation.FuncAnimation(fig=fig,
                              func=animate,
                              init_func=init,
                              frames=NTIME,
                              interval=100,
                              blit=True)

HTML(ani.to_jshtml())

---
## 3.5 What's next ?

--> If you want to see what the simulation looks like when run for a longer time than 5 days, you can have a look at the 2-month simulation we stored on the SASIP storage (accessible here through `/data/`):

In Cell 14 above, you can replace:

```xr.open_dataset(dirwo+'/output_simudyn_5d.nc',group='data')``` 

by :

```xr.open_dataset('/data/model-outputs/NeXtSIM-DG/demo-testcases/output_simudyn_2mo.nc',group='data')```

and re-run the plot cells.